# Very naive learning on the vector of surrounding bases with standardish machine learning algorithms from scikit sklearn

---

### Analysis

I will try to naively predict the mehtylation value (0,1) based on naive sequence context feauture vectors.

I will use a very popular "state of the art" general machine learning library: xgboost.
    



### Conlcusions

- It gains a slight, but real increse in precision over the other standard models (3-4%)

---

In [1]:
import numpy as np

import time
import xgboost as xgb

import sys
sys.path.append('../my_modules')
from loading_utils import read_my_data,create_sets

#workdir='/nagyvinyok/adat84/sotejedlik/ribli/methylation_code/modelling'
#subprocess.call(['mkdir',workdir])
#os.chdir(workdir)

Couldn't import dot_parser, loading of dot files will not be possible.


### Load data

In [2]:
x,y= read_my_data(fname='../prepare_data/small_meth_feat_vect.csv')

#select the whole dataset
(train_x,train_y),(valid_x,valid_y),(test_x,test_y)= create_sets(
    x,y,N_train=80000,N_valid=10000,N_test=10000,length=2000)

Loading data... 


#### Create xgboost matrices

In [3]:
dtrain = xgb.DMatrix( train_x, label=train_y)
dvalid = xgb.DMatrix( valid_x, label=valid_y)
dtest = xgb.DMatrix( test_x, label=test_y)

#### Define params

In [4]:
param = {'bst:max_depth':5,
         'bst:eta':0.01,
         'min_child_weight':1,
         'colsample_bytree':1,
         'silent':1,
         'objective': "binary:logistic",
         'eval_metric': 'error',
         'nthread':4}

evallist  = [(dvalid,'eval'), (dtrain,'train')]

#### Fit

- The evaluation set is used to monitor the learning

In [5]:
#train
num_round = 1000
bst = xgb.train(param,
                dtrain,
                evals=evallist,
                num_boost_round=num_round)
                #early_stopping_rounds=5,
                #feval=feval) #not using my own evaluation function because its sloW!

train_bst_pred=bst.predict(dtrain)
test_bst_pred=bst.predict(dvalid)

[0]	eval-error:0.400300	train-error:0.403462
[1]	eval-error:0.397200	train-error:0.400263
[2]	eval-error:0.392800	train-error:0.394112
[3]	eval-error:0.393200	train-error:0.393475
[4]	eval-error:0.389700	train-error:0.389575
[5]	eval-error:0.375200	train-error:0.377275
[6]	eval-error:0.377000	train-error:0.377250
[7]	eval-error:0.374800	train-error:0.370875
[8]	eval-error:0.371300	train-error:0.369987
[9]	eval-error:0.366900	train-error:0.366163
[10]	eval-error:0.363800	train-error:0.360013
[11]	eval-error:0.359800	train-error:0.352463
[12]	eval-error:0.360500	train-error:0.354650
[13]	eval-error:0.356500	train-error:0.350875
[14]	eval-error:0.350800	train-error:0.346850
[15]	eval-error:0.346500	train-error:0.341213
[16]	eval-error:0.341400	train-error:0.338987
[17]	eval-error:0.344000	train-error:0.337725
[18]	eval-error:0.339500	train-error:0.334225
[19]	eval-error:0.338100	train-error:0.331863
[20]	eval-error:0.332400	train-error:0.327312
[21]	eval-error:0.331600	train-error:0.32663

#### Final scores

In [6]:
print 'train score:',list(map(round,bst.predict(dtrain))==train_y).count(True)/float(len(train_y))
print 'validation score:',list(map(round,bst.predict(dvalid))==valid_y).count(True)/float(len(valid_y))
print 'test score:',list(map(round,bst.predict(dtest))==test_y).count(True)/float(len(test_y))

train score: 0.811025
validation score: 0.7701
test score: 0.7731
